In [2]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import pandas as pd
import gc
import re
from qp_utils import (
    expand_vars, load_year, save_compact_meta,
    decode_labels, show_value_table_json, peek_sav_columns,
)

RAW_DIR = Path("Spss")
BASE_YEAR = 2010
years = list(range(2010, 2023+1))
files = {}
for y in years:
    p = RAW_DIR / f"QP_Estabelecimentos_{y}.sav"
    if p.exists():
        files[y] = p

print()
def strip_year_tag_any(name: str) -> str:
    s = re.sub(r'_(\d{2})(?=(_|$))', '', name)
    s = re.sub(r'(?<=\D)(\d{2})$', '', s)
    return s
def base_vars_from_sav(path: Path) -> set[str]:
    cols = peek_sav_columns(path)["var"].tolist()
    return {strip_year_tag_any(v) for v in cols}

vars_by_year: dict[int, set[str]] = {}
for y, p in sorted(files.items()):
    s = base_vars_from_sav(p)
    vars_by_year[y] = s
    print(f"Year {y}: {len(s)} base variables in {p.name}")


def base_vars_from_sav(path: Path) -> set[str]:
    cols = peek_sav_columns(path)["var"].tolist()
    return {strip_year_tag_any(v) for v in cols}

def build_presence_table(files: dict[int, Path], base_year: int, diff_only: bool = True):
    if base_year not in files:
        raise ValueError(f"base_year {base_year} not in files")

    # Build sets of base variable names per year
    vars_by_year: dict[int, set[str]] = {y: base_vars_from_sav(p) for y, p in files.items()}

    # Union of all variable base-names, so we see extras not in base too
    all_vars = sorted(set().union(*vars_by_year.values()))

    # Build presence matrix
    years_sorted = [base_year] + sorted([y for y in files if y != base_year])
    data = {
        y: [v in vars_by_year[y] for v in all_vars]
        for y in years_sorted
    }
    presence = pd.DataFrame(data, index=all_vars)

    # Optionally filter to differences vs base column
    base_col = years_sorted[0]
    if diff_only:
        mask = (presence.T != presence[base_col]).any()
        presence = presence.loc[mask]

    return presence

presence_df = build_presence_table(files, BASE_YEAR, diff_only=True)

# See the raw boolean table (if you want)
presence_df.head(40)

# wanted_tpl = ["ano_{yy}","nuemp_{yy}","nuest_{yy}","ntrab_{yy}","sexo_{yy}","idade_{yy}_TB_COD","dtadm_{yy}"]

# df_2008, meta_2008 = load_year(2008, files[2008], cols=expand_vars(wanted_tpl, 2008))
# df_2009, meta_2009 = load_year(2009, files[2009], cols=expand_vars(wanted_tpl, 2009))

# save_compact_meta(meta_2008, 2008, df_2008)
# save_compact_meta(meta_2009, 2009, df_2009)

# df = pd.concat([df_2008, df_2009], ignore_index=True, copy=False)
# del df_2008, df_2009, meta_2008, meta_2009; gc.collect()

# # decode labels where available (e.g., sexo)
# decode_labels(df, "sexo")

# # quick table for 2009
# show_value_table_json(df, "sexo", 2009, sort_index=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

Year 2010: 18 base variables in QP_Estabelecimentos_2010.sav
Year 2011: 18 base variables in QP_Estabelecimentos_2011.sav
Year 2012: 18 base variables in QP_Estabelecimentos_2012.sav
Year 2013: 18 base variables in QP_Estabelecimentos_2013.sav
Year 2014: 18 base variables in QP_Estabelecimentos_2014.sav
Year 2015: 18 base variables in QP_Estabelecimentos_2015.sav
Year 2016: 18 base variables in QP_Estabelecimentos_2016.sav
Year 2017: 18 base variables in QP_Estabelecimentos_2017.sav
Year 2018: 18 base variables in QP_Estabelecimentos_2018.sav
Year 2019: 18 base variables in QP_Estabelecimentos_2019.sav
Year 2020: 18 base variables in QP_Estabelecimentos_2020.sav
Year 2021: 18 base variables in QP_Estabelecimentos_2021.sav
Year 2022: 21 base variables in QP_Estabelecimentos_2022.sav
Year 2023: 21 base variables in QP_Estabelecimentos_2023.sav


,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
CAE2,True,True,True,True,True,True,True,True,True,True,False,False,False,False
CAE2_EST,False,False,False,False,False,False,False,False,False,False,False,True,True,False
CAE2_est,False,False,False,False,False,False,False,False,False,False,True,False,False,False
CAEST1L,False,False,False,False,False,False,False,False,False,False,False,False,False,True
CAEST2,False,False,False,False,False,False,False,False,False,False,False,False,False,True
EMP_ID,True,True,True,True,True,True,True,True,True,True,True,True,True,False
ESCDIM1EST,False,False,False,False,False,False,False,False,False,False,False,False,False,True
ESCDIM2EST,False,False,False,False,False,False,False,False,False,False,False,False,False,True
ESCDIMEST_LINHAS1,False,False,False,False,False,False,False,False,False,False,False,False,False,True
ESCDIMEST_LINHAS2,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [6]:
presence_df[[2009, 2008]].head(40)

,2009,2008
caes1,False,False
caes1l,False,False
caes2,False,False
caes2l,False,False
caes3,False,False
caest1l,True,True
caest2,True,True
caest3,True,True
escdim1desest,False,False
escdim1est,False,True


In [14]:
peek_sav_columns(files[2010])

,var,label,spss_type
0,ANO,Ano de referência,
1,NPC_FIC,Número de Identificação Ficticio da empresa,
2,NUEMP,Número da empresa (ligação com a série QP até ...,
3,EMP_ID,ID da empresa (ano 2010 e seguintes),
4,NUEST,Número do estabelecimento (ligação com a série...,
5,ESTAB_ID,ID do estabelecimento (ano 2010 e seguintes),
6,sede,Estabelecimento sede,
7,nut1_est,NUT I do estabelecimento em 31 de outubro,
8,nut2_est,NUT II do estabelecimento em 31 de outubro,
9,caest1l,Actividade Económica do estabelecimento (CAE_R...,


In [22]:
peek_sav_columns(files[2023])

,var,label,spss_type
0,ANO,Ano de referência,
1,NPC_FIC,Número de Identificação Fictício da empresa,
2,NUEMP,Número da empresa (ligação com a série QP até ...,
3,N_EMP,ID da empresa (ano 2010 e seguintes),
4,NUEST,Número do estabelecimento (ligação com a série...,
5,N_ESTAB,ID do estabelecimento (ano 2010 e seguintes),
6,SEDE,Estabelecimento sede,
7,PAIS,País do estabelecimento,
8,NUT1_ESTAB2013,NUT I ddo estabelecimento em 31 de outubro (NU...,
9,NUT2_ESTAB2013,NUT II do estabelecimento em 31 de outubro (NU...,
